# Movie Recommendation (Movielens)
- 유저가 영화에 대해 평점을 매긴 데이터 (MovieLens 1M Dataset)
```
wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
mv ml-1m.zip ~/aiffel/recommendata_iu/data
unzip ml-1m.zip
```
- 방법 : 별점을 시청횟수로 해석해서 생각하겠습니다. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 데이터 준비

In [ ]:
import pandas as pd
import numpy as np
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

In [ ]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

In [ ]:
# rating 컬럼의 이름을 count로 바꿉니다.
#ratings.rename(columns={'rating':'count'}, inplace=True)

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

## 데이터 탐색

In [ ]:
# Ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

In [ ]:
# Ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

In [ ]:
ratings.head()

In [ ]:
pd.merge([ratings, movies], join='outer', axis=1, join_axes='movie_id

In [ ]:
# 가장 인기있는 영화 30개 (인기순)
movie_popular = ratings.groupby('movie_id')['user_id'].count()
movie_popular

In [ ]:
ratings.tail()

## 내가 선호하는 영화를 5가지 골라서 rating에 추가하기

In [ ]:
movies[movies['genre']=='Animation']
movies[movies['genre']=='Comedy']
ratings.head()

In [ ]:
# 내가 좋아하는 영화. 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = [0, 720, 2120, 3924, 248]

# 맨 마지막보다 하나 많은 user_id가 위 영화를 5점씩 rating했다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'rating':[5]*5, 'timestamp':[0]*5 })

if not ratings.isin({'user_id':['6041']})['user_id'].any():  # user_id에 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

## CSR matrix 만들기

In [ ]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].max()
num_movie = ratings['movie_id'].max()



print(num_user)
print(num_movie)
print(ratings.rating.shape)
print(ratings.movie_id.shape)
print(ratings.user_id.shape)

csr_data = csr_matrix((ratings.rating, (ratings.user_id, ratings.movie_id)), shape= (num_user+1, num_movie+1))
csr_data

## 모델 훈련하기

In [ ]:
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=10000, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

In [ ]:
als_model.fit(csr_data_transpose)

In [ ]:
me = 6041
toy_story = 0
me_vector, toy_story_vector = als_model.user_factors[me], als_model.item_factors[0]


In [ ]:
me_vector

In [ ]:
toy_story_vector

In [ ]:
np.dot(me_vector, toy_story_vector)

In [ ]:
## 나의 선호도 파악하기

In [ ]:
movies.head()


In [ ]:
movies[movies['movie_id']==211].title

In [ ]:
BrowningVersion = 211
BrowningVersion_vector = als_model.item_factors[BrowningVersion]
np.dot(me_vector, BrowningVersion_vector)

In [ ]:
## 내가 좋아하는 영화와 비슷한 영화 추천받기

In [ ]:
movie_unique = movies["title"].unique()
movie_to_idx = {v:k for k, v in enumerate(movie_unique)}
favorite_movie = 'Jumanji (1995)' # toy story
favorite_movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(favorite_movie_id, N=15)
similar_movie

In [ ]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [ ]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [ ]:
get_similar_movie('Jumanji (1995)')

## 내가 가장 좋아할 만한 영화 추천 받기

#### user = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

In [255]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Two or Three Things I Know About Her (1966)',
 'Vertigo (1958)',
 'Big Trees, The (1952)',
 'Couch in New York, A (1996)',
 'Open Season (1996)',
 'Vermont Is For Lovers (1992)',
 'Leather Jacket Love Story (1997)',
 'Beneath the Planet of the Apes (1970)',
 'Hocus Pocus (1993)',
 'Red Corner (1997)',
 'Julien Donkey-Boy (1999)',
 'They Bite (1996)',
 'Two Deaths (1995)',
 'Saludos Amigos (1943)',
 'King of Marvin Gardens, The (1972)',
 'Return of the Fly (1959)',
 'Spring Fever USA (a.k.a. Lauderdale) (1989)',
 'Jade (1995)',
 'Stonewall (1995)',
 'Bell, Book and Candle (1958)']

### 느낀점
- 너무 어렵다. 특히, 두 개의 테이블을 합쳐서 한 눈에 보고 싶었는데, 시도한 대로 바로 되지 않았다. 다시 한 번 시도할 필요가 있을 듯 하다..